In [65]:
import pandas as pd;
import numpy as np;
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK
from hyperopt import hp

MAX_EVALS = 500
N_FOLDS = 10



Hyperopt

Hyperopt is one of several automated hyperparameter tuning libraries using Bayesian optimization. These libraries differ in the algorithm used to both construct the surrogate (probability model) of the objective function and choose the next hyperparameters to evaluate in the objective function. Hyperopt uses the Tree Parzen Estimator (TPE). Other Python libraries include Spearmint, which uses a Gaussian process for the surrogate, and SMAC, which uses a random forest regression.

Hyperopt has a simple syntax for structuring an optimization problem which extends beyond hyperparameter tuning to any problem that involves minimizing a function. Moreover, the structure of a Bayesian Optimization problem is similar across the libraries, with the major differences coming in the syntax (and in the algorithms behind the scenes that we do not have to deal with).


In [55]:
# import Dataset to play with it
train= pd.read_csv("/home/altieris/datascience/data/santander-customer-transaction-prediction/train.csv")

In [69]:
cols=["target","ID_code"]

drop_from_shap=['var_185', 'var_181', 'var_182', 'var_183', 'var_14', 'var_41',
       'var_98', 'var_103', 'var_189', 'var_10', 'var_7', 'var_64',
       'var_65', 'var_84', 'var_73', 'var_79', 'var_96', 'var_19',
       'var_17', 'var_160', 'var_143', 'var_46', 'var_153', 'var_39',
       'var_38', 'var_37', 'var_158', 'var_136', 'var_161', 'var_42',
       'var_129', 'var_30', 'var_29', 'var_126', 'var_124', 'var_27',
       'var_117', 'var_59', 'var_100','var_168', 'var_3', 'var_8', 'var_74', 'var_72', 'var_178',
       'var_54', 'var_171', 'var_50', 'var_152', 'var_55', 'var_140',
       'var_138', 'var_193', 'var_156', 'var_113', 'var_63', 'var_15',
       'var_60', 'var_69', 'var_159', 'var_47', 'var_61', 'var_176',
       'var_68', 'var_16', 'var_57', 'var_101', 'var_25', 'var_120','var_194', 'var_11', 'var_45', 'var_196', 'var_83', 'var_20',
       'var_77', 'var_134', 'var_82', 'var_102', 'var_97', 'var_187',
       'var_142', 'var_88', 'var_62', 'var_4']

train_x = train.drop(cols,axis=1).drop(drop_from_shap,axis=1)
train_y = train["target"]

train_set = lgb.Dataset(train_x, label=train_y)


In [57]:
import csv
from hyperopt import STATUS_OK
from timeit import default_timer as timer

def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, early_stopping_rounds = 100, metrics = 'auc', seed = 50)
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

In [58]:
# Define the search space
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', 
                               [{'boosting_type': 'gbdt', 
                                    'subsample': hp.uniform('gdbt_subsample', 0.5, 1)}, 
                                 {'boosting_type': 'dart', 
                                     'subsample': hp.uniform('dart_subsample', 0.5, 1)},
                                 {'boosting_type': 'goss'}]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0)
}

In [63]:
from hyperopt import tpe

# optimization algorithm
tpe_algorithm = tpe.suggest

In [59]:
from hyperopt import Trials

# Keep track of results
bayes_trials = Trials()

In [60]:
# File to save first results
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

In [61]:
from hyperopt import fmin

In [70]:
%%capture

# Global variable
global  ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(50))

KeyboardInterrupt: 

In [71]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results[:2]

KeyError: 'loss'